In [41]:
# %load_ext autoreload
# %autoreload 2
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from ThreeDCNN_LSTM import CNNLSTM, train
import torch.optim as optim
import torch.nn as nn
from NeuroimagingDataset import NeuroimagingDataset
import numpy as np

# Configuration options
# Define the number of folds and batch size
k_folds = 5
batch_size = 16

# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the k-fold cross validation
kf = KFold(n_splits=k_folds, shuffle=True)

train_dataset = NeuroimagingDataset('/Users/ejzhang/Downloads/NDMIC/ResultsS',
                                    '/Users/ejzhang/Downloads/NDMIC/phenotypic_data.csv',
                                    feature_types='ALFF')

In [42]:
# Initialize the model and optimizer
model = CNNLSTM().to(device)
#optimizer = optim.Adam(model.parameters(), lr=0.0005)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
)
    
# Train the model on the current fold
N_EPOCHS=100
train(model, device, train_loader, optimizer, criterion, N_EPOCHS, True)


Epoch: 0

Epoch: 1

Epoch: 2

Epoch: 3

Epoch: 4

Epoch: 5

Epoch: 6

Epoch: 7

Epoch: 8

Epoch: 9

Epoch: 10

Epoch: 11

Epoch: 12

Epoch: 13

Epoch: 14

Epoch: 15

Epoch: 16

Epoch: 17

Epoch: 18

Epoch: 19

Epoch: 20

Epoch: 21

Epoch: 22

Epoch: 23

Epoch: 24

Epoch: 25

Epoch: 26

Epoch: 27

Epoch: 28

Epoch: 29

Epoch: 30

Epoch: 31

Epoch: 32

Epoch: 33

Epoch: 34

Epoch: 35

Epoch: 36

Epoch: 37

Epoch: 38

Epoch: 39

Epoch: 40

Epoch: 41

Epoch: 42

Epoch: 43

Epoch: 44

Epoch: 45

Epoch: 46

Epoch: 47

Epoch: 48

Epoch: 49

Epoch: 50

Epoch: 51

Epoch: 52

Epoch: 53

Epoch: 54

Epoch: 55

Epoch: 56

Epoch: 57

Epoch: 58

Epoch: 59

Epoch: 60

Epoch: 61

Epoch: 62

Epoch: 63

Epoch: 64

Epoch: 65

Epoch: 66

Epoch: 67

Epoch: 68

Epoch: 69

Epoch: 70

Epoch: 71

Epoch: 72

Epoch: 73

Epoch: 74

Epoch: 75

Epoch: 76

Epoch: 77

Epoch: 78

Epoch: 79

Epoch: 80

Epoch: 81

Epoch: 82

Epoch: 83

Epoch: 84

Epoch: 85

Epoch: 86

Epoch: 87

Epoch: 88

Epoch: 89

Epoch: 90

Epoch: 9

In [43]:
# Loop through each fold
for fold, (train_idx, test_idx) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold + 1}")
    print("-------")
    
    # get rid of bad indices
    train_idx = np.array([idx for idx in train_idx if idx not in [70, 83]])
    test_idx = np.array([idx for idx in test_idx if idx not in [70, 83]])

    # Define the data loaders for the current fold
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(train_idx),
    )
    test_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(test_idx),
    )

    # Initialize the model and optimizer
    model = CNNLSTM().to(device)
    model.load_state_dict(torch.load("model_checkpoint_e"))
    model.eval()

    # Train the model on the current fold
    N_EPOCHS=50
    train(model, device, train_loader, optimizer, criterion, N_EPOCHS, False)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0
    correct = 0
    size = 0
    with torch.no_grad():
        for data, target in test_loader:
            size += len(data)
            data = data.unsqueeze(1) # Unsqueeze for consistency!
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= size
    accuracy = 100.0 * correct / size

    # Print the results for the current fold
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{size} ({accuracy:.2f}%)\n")

Fold 1
-------

Epoch: 0

Epoch: 1

Epoch: 2

Epoch: 3

Epoch: 4

Epoch: 5

Epoch: 6

Epoch: 7

Epoch: 8

Epoch: 9

Epoch: 10

Epoch: 11

Epoch: 12

Epoch: 13

Epoch: 14

Epoch: 15

Epoch: 16

Epoch: 17

Epoch: 18

Epoch: 19

Epoch: 20

Epoch: 21

Epoch: 22

Epoch: 23

Epoch: 24

Epoch: 25

Epoch: 26

Epoch: 27

Epoch: 28

Epoch: 29

Epoch: 30

Epoch: 31

Epoch: 32

Epoch: 33

Epoch: 34

Epoch: 35

Epoch: 36

Epoch: 37

Epoch: 38

Epoch: 39

Epoch: 40

Epoch: 41

Epoch: 42

Epoch: 43

Epoch: 44

Epoch: 45

Epoch: 46

Epoch: 47

Epoch: 48

Epoch: 49
Test set: Average loss: 0.0400, Accuracy: 21/29 (72.41%)

Fold 2
-------

Epoch: 0

Epoch: 1

Epoch: 2

Epoch: 3

Epoch: 4

Epoch: 5

Epoch: 6

Epoch: 7

Epoch: 8

Epoch: 9

Epoch: 10

Epoch: 11

Epoch: 12

Epoch: 13

Epoch: 14

Epoch: 15

Epoch: 16

Epoch: 17

Epoch: 18

Epoch: 19

Epoch: 20

Epoch: 21

Epoch: 22

Epoch: 23

Epoch: 24

Epoch: 25

Epoch: 26

Epoch: 27

Epoch: 28

Epoch: 29

Epoch: 30

Epoch: 31

Epoch: 32

Epoch: 33

Epoch: 

KeyboardInterrupt: 